In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Pre-procesamiento

- Descargar los datos https://drive.google.com/drive/folders/12EbE0PEEnCjUVjhzMDNa1W7M9tR4ZzwW?usp=share_link
- Hacer merge de los datos
```bash
awk -F ';' 'FNR==1 && NR!=1{next} {print}' *.csv | sed '/^$/d' > ../merged.csv
```

Diccionario de datos facilitado por [https://github.com/luchobenitez/monda-guasu](https://github.com/luchobenitez/monda-guasu)

|Campo|Descripción|Tipo de Dato|
|-----|-----------|------------|
|serialtarjeta|Identificador de la tarjeta. Puede ser nominal o no, por lo que se puede llegar a la identidad del pasajero. Para nuestro análisis asumimos que es un pasajero|hash md5|
|idsam|Identificador del chip instalado en el equipo verificador. No necesariamente permanece con el mismo equipo verificador. Puede cambiar de bus, inclusive de línea o de EOT. Asumimos que identifica al bus|string|
|fechahoraevento|Timestamp del inicio del viaje|timestamp|
|producto|Tipo de producto donde:  - CR. Crédito o saldo negativo de la tarjeta - ES. Uso Especial (Estudiante o discapacitado) - MO. Monedero o uso normal|string|
|montoevento|Monto del evento descontado de la tarjeta. |entero|
|consecutivoevento|Identificador consecutivo de numero de transacción realizado con la tarjeta identificada con serialtarjeta|entero|
|identidad|Se refiere al propietario del validador donde: - 1. VMT - 2. MAS - 3. JAHA |entero|
|tipoevento|Tipo de evento donde: - 4. Viaje Normal - 8. Devolución - 10. Recarga de la tarjeta. - 14. Devolucion. Utilizaremos solo los del tipo 4|punto flotante|
|longitude|Longitud geográfica|punto flotante|
|Latitude|Latitude geográfica|punto flotante|
|idrutaestacion|Identificador de ruta asociado a una EOT. Ver tabla adicional|string|
|tipotransporte|Tipo de transporte donde - 0. Bus municipal interno - 1. Normal  - 3. Diferencial|entero|

In [2]:
df = pl.read_csv('merged.csv', separator=';', try_parse_dates=True, columns=['serialtarjeta','consecutivoevento'])
# order by consecutivoevento descending and show 20 rows
df = df.sort('consecutivoevento')
df

serialtarjeta,consecutivoevento
str,i64
"""37ce879fc894a9...",2
"""1228002de737cc...",2
"""6143e4cd392b4d...",2
"""b59787e0e5dd50...",2
"""437cadb2126cd0...",2
"""6c5abeb0144b37...",2
"""2db42068576860...",2
"""e5e07438d46c57...",2
"""3123c3642ac0c1...",2


## Quiero saber cuantas veces hay un numero consecutivo faltante en la columna consecutivoevento. 
Pasos a seguir:
- Agrupar por serialtarjeta
- calcular la diferencia entre los numeros consecutivos
- contar cuantas veces ocurre cada una de las diferencias

In [8]:
# Using polars group df by serialtarjeta and then calculate the difference between the consecutive events in consecutivoevento and count the number of events. 
missing_consecutivoevento_por_serialtarjeta = df.groupby(['serialtarjeta']).agg([pl.col('consecutivoevento').diff().count()])
# rename consecutivoevento column to consecutivoevento_diff_count
missing_consecutivoevento_por_serialtarjeta = missing_consecutivoevento_por_serialtarjeta.rename({'consecutivoevento': 'consecutivoevento_diff_count'})
missing_consecutivoevento_por_serialtarjeta.head()

serialtarjeta,consecutivoevento_diff_count
str,u32
"""cf47f49b8b4e62...",55
"""df34af489dc0c5...",10
"""6679d49f54df5b...",53
"""6e5f25c3f14f44...",9
"""d7fe7c90679bd1...",167


In [9]:
# oder missing_consecutivoevento_por_serialtarjeta by consecutivoevento descending and show 20 rows
missing_consecutivoevento_por_serialtarjeta.sort('consecutivoevento_diff_count', reverse=True).head(20).to_numpy()

C:\Users\Karl\AppData\Local\Temp/ipykernel_27032/2919029341.py:2: DeprecationWarning: `reverse` is deprecated as an argument to `sort`; use `descending` instead.
  missing_consecutivoevento_por_serialtarjeta.sort('consecutivoevento_diff_count', reverse=True).head(20).to_numpy()


array([['18f6e00ec04d8e05438254c3b1e8502b', 22308],
       ['bc069bee563eaca48c54a33f788b0f27', 18562],
       ['9ab72b0115d5793f64285348af916464', 16054],
       ['0a0d03a58ca2cee8d5de631884cf6089', 15285],
       ['0e13796ff34ba4016a16cff9dd069430', 14776],
       ['21a4ee1cf5379525e614640fb14f23f3', 14743],
       ['ad1ce39e55470a064263da9209b7d4d6', 14021],
       ['adf929e7c06a0a79fdff9b49ce78b08d', 13959],
       ['f6ef359bc074b3137c4ee70fa475d3b1', 13344],
       ['c6ec60f6ff89eb3716510a1f08ab11d8', 12261],
       ['36c597c0a77ebcdc38811c3dcd1a124c', 11292],
       ['e1a9b54899db863091a64bc878855073', 10691],
       ['311cf4ea8b9c56612f986cb7f53c666c', 10448],
       ['c9ba98eea1530eb9a55aff3d27ad49d2', 10401],
       ['d9ddc0ef08b376d1e92ba38037c05e19', 9675],
       ['cfd016a6423f0071a9bb8d1dcbe3829a', 9178],
       ['02279ac6de68db080dcfec5348f7cec2', 8993],
       ['6c034eaaec3b846aebba7b25c69f7103', 8825],
       ['a8241cdb085eb3dc53de1e5d3e533b2d', 8245],
       ['b66d8adc

In [10]:
# Cross validiate by filtering serialtarjeta = 18f6e00ec04d8e05438254c3b1e8502b and count the differences of consecutivoevento
df.filter(pl.col("serialtarjeta") == '18f6e00ec04d8e05438254c3b1e8502b').groupby(['serialtarjeta']).agg([pl.col('consecutivoevento').diff().count()])

serialtarjeta,consecutivoevento
str,u32
"""18f6e00ec04d8e...",22308


In [12]:
# order by consecutivoevento_diff_count in descending order and save a csv of missing_consecutivoevento_por_serialtarjeta
missing_consecutivoevento_por_serialtarjeta.sort('consecutivoevento_diff_count', reverse=True).to_pandas().to_csv('missing_consecutivoevento_por_serialtarjeta.csv')

C:\Users\Karl\AppData\Local\Temp/ipykernel_27032/2579270040.py:2: DeprecationWarning: `reverse` is deprecated as an argument to `sort`; use `descending` instead.
  missing_consecutivoevento_por_serialtarjeta.sort('consecutivoevento_diff_count', reverse=True).to_pandas().to_csv('missing_consecutivoevento_por_serialtarjeta.csv')
